## Start the beam-flink job server

In [1]:
from hops import beam as hops_beam
# Start Beam jobservice
hops_beam.start(taskmanager_heap_size=8192)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
47,application_1566305877532_0013,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.

Job server log is available at:/srv/hops/hadoop/logs/userlogs/6cNHepU-KQfhT5sP9H_vEYwpMAgs7wZjeSs0hav8V-U/application_1566305877532_0013/container_e05_1566305877532_0013_01_000001/beamjobserver-flink_tutorial-runner-35232.log

Job host:ip-172-31-10-231.eu-north-1.compute.internal
Job port:35232
{'job_port': 35232, 'job_host': 'ip-172-31-10-231.eu-north-1.compute.internal', 'jobserver.pid': 29417, 'artifact_port': 10685, 'expansion_port': 57278}

## TFMA Notebook example

This notebook describes how to export your model for TFMA and demonstrates the analysis tooling it offers.

Note: Please make sure to follow the instructions in [README.md](https://github.com/tensorflow/tfx/blob/master/tfx/examples/chicago_taxi/README.md) when running this notebook

## Setup

Import necessary packages.

In [2]:
import apache_beam as beam
from hops import beam as hops_beam
import os
from tfx.examples.chicago_taxi import preprocess
import shutil
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from google.protobuf import text_format 
from tensorflow.python.lib.io import file_io
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.coders import example_proto_coder
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import schema_utils
from tfx.examples.chicago_taxi.trainer import task
from tfx.examples.chicago_taxi.trainer import taxi
from hops import hdfs as hopsfs
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions, SetupOptions, HadoopFileSystemOptions, PortableOptions, WorkerOptions, DebugOptions

Helper functions and some constants for running the notebook locally.

In [3]:
BASE_DIR = hopsfs.project_path(exclude_nn_addr=True)
DATA_DIR = os.path.join(BASE_DIR, 'Resources/data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'Resources/taxi_out')
TMP_DIR = os.path.join(BASE_DIR, 'Resources/taxi_tmp')

# Base dir containing train and eval data
TRAIN_DATA_DIR = os.path.join(DATA_DIR, 'train')
EVAL_DATA_DIR = os.path.join(DATA_DIR, 'eval')

# Base dir where TFT writes training data
TFT_TRAIN_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_train')
TFT_TRAIN_FILE_PREFIX = 'train_transformed'

# Base dir where TFT writes eval data
TFT_EVAL_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_eval')
TFT_EVAL_FILE_PREFIX = 'eval_transformed'

TF_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tf')

# Base dir where TFMA writes eval data
TFMA_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tfma')

SERVING_MODEL_DIR = 'serving_model_dir'
EVAL_MODEL_DIR = 'eval_model_dir'


def get_tft_train_output_dir(run_id):
    return _get_output_dir(TFT_TRAIN_OUTPUT_BASE_DIR, run_id)


def get_tft_eval_output_dir(run_id):
    return _get_output_dir(TFT_EVAL_OUTPUT_BASE_DIR, run_id)


def get_tf_output_dir(run_id):
    return _get_output_dir(TF_OUTPUT_BASE_DIR, run_id)

def get_tfma_output_dir(run_id):
    return _get_output_dir(TFMA_OUTPUT_BASE_DIR, run_id)

def _get_output_dir(base_dir, run_id):
    return os.path.join(base_dir, 'run_' + str(run_id))

def get_schema_file():
    return os.path.join(OUTPUT_DIR, 'schema.pbtxt')


Configure the beam pipeline

In [4]:
pipeline_args = hops_beam.get_portable_runner_config()
options=PipelineOptions(flags=pipeline_args)

## Compute statistics over transformed data 


In [5]:
# Compute stats over transformed training data.
TRANSFORMED_TRAIN_DATA = os.path.join(get_tft_train_output_dir(0), TFT_TRAIN_FILE_PREFIX + "*") 
transformed_train_stats = tfdv.generate_statistics_from_tfrecord(data_location=TRANSFORMED_TRAIN_DATA, output_path=os.path.join(TMP_DIR, 'transformed_train_stats'), pipeline_options=options)

In [6]:
# Visualize transformed training data stats and compare to raw training data. 
# Use 'Feature search' to focus on a feature and see statistics pre- and post-transformation.
train_stats = tfdv.load_statistics(os.path.join(TMP_DIR,'train_stats'))
tfdv.visualize_statistics(transformed_train_stats, train_stats, lhs_name='TRANSFORMED', rhs_name='RAW')

<IPython.core.display.HTML object>